In [1]:
!pip install pyspark

In [2]:
import requests
import json
import pandas as pd
from pyspark.sql import HiveContext, Row, SparkSession

In [3]:
spark = SparkSession.builder.appName("HiveContext_SpaceX").enableHiveSupport().getOrCreate()
hiveCtx = HiveContext(spark.sparkContext)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:731: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# Fetch SpaceX API data

url = "https://api.spacexdata.com/v5/launches"
response = requests.get(url)
data = response.json()

In [5]:
# Filter out the data lunch successful
successful = [launch for launch in data if launch.get("success") == True]

# Print first 2 observation
successful[:2]

[{'fairings': {'reused': False,
   'recovery_attempt': False,
   'recovered': False,
   'ships': []},
  'links': {'patch': {'small': 'https://images2.imgbox.com/95/39/sRqN7rsv_o.png',
    'large': 'https://images2.imgbox.com/a3/99/qswRYzE8_o.png'},
   'reddit': {'campaign': None,
    'launch': None,
    'media': None,
    'recovery': None},
   'flickr': {'small': [], 'original': []},
   'presskit': None,
   'webcast': 'https://www.youtube.com/watch?v=dLQ2tZEH6G0',
   'youtube_id': 'dLQ2tZEH6G0',
   'article': 'https://en.wikipedia.org/wiki/Ratsat',
   'wikipedia': 'https://en.wikipedia.org/wiki/Ratsat'},
  'static_fire_date_utc': '2008-09-20T00:00:00.000Z',
  'static_fire_date_unix': 1221868800,
  'net': False,
  'window': 0,
  'rocket': '5e9d0d95eda69955f709d1eb',
  'success': True,
  'failures': [],
  'details': 'Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the\xa0SpaceX Falcon 1',
  'crew':

In [6]:
with open("successful_launches.json", "w") as f:
    json.dump(successful, f)

In [7]:
# Load Json file into PySpark df
df = hiveCtx.read.json("successful_launches.json")

In [8]:
# Register df as a table
df.registerTempTable("success_table")

/usr/local/lib/python3.12/dist-packages/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### SQL Query:
**Count and display the number of successful launches**

In [9]:
# Query with HiveQL
result = hiveCtx.sql("SELECT COUNT(*) AS SuccessCount FROM success_table")
result.show()

+------------+
|SuccessCount|
+------------+
|         181|
+------------+

